In [ ]:
from dotenv import load_dotenv
# import streamlit as st
# from PyPDF2 import PdfReader
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
# from langchain.callbacks import get_openai_callback

In [ ]:
load_dotenv()

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0.9,request_timeout=300)

    ################################## GET MODULES ##############################################################
    ################# chain1: Topic name(only 5 for now)

module_prompt = PromptTemplate(
    input_variables=["topic", "application", "module_number", "level_dropdown"],
    template="You want to create a {module_number} module {level_dropdown} level personalized course on {topic} which i will use for {application}. Let's start by designing the module structure and subtopics to include in each module. Write in this format: Module 1 heading then subtopic 1 heading ,subtopic 2 heading,subtopic 3 heading......... give numbering to module and subtopics.Don't write any NOTE in the end just write module name and subtopic name. Give the output in JSON format with key value pairs. The format will be Module name then its heading as value then subkey would be all the subtopics in that module and their heading as values."
)



module_chain = LLMChain(llm=llm, prompt=module_prompt)

module_output =  module_chain.predict(topic='Large language model', application = 'academic exam', module_number = 2, level_dropdown= 'beginner')



In [ ]:
print(module_output)

In [ ]:

# Parse the JSON data
data = json.loads(module_output)

# Access Module 1 and Subtopic 3
module_1 = data["Module 1"]
subtopic_3 = module_1["Subtopics"]["3"]

# Print the results
print("Module 1:", module_1["Heading"])
print("Subtopic 3:", subtopic_3)


In [ ]:
#save json data
with open("module_output.json", "w") as file:
    file.write(module_output)


create content for each module and subtopic

In [ ]:
# print(module_output)

data = json.loads(module_output)

for x in data.values():
    print(x)

In [ ]:
import json
import concurrent.futures

# Define the content_prompt template
content_prompt = PromptTemplate(
    input_variables=["subtopic"],
    template="Give me a detailed description of {subtopic}. Write in bulleted form and make sure to use code examples so that any beginner-level student can understand."
)

# Create the LLMChain instance
content_chain = LLMChain(llm=llm, prompt=content_prompt)

data = json.loads(module_output)

# Function to generate content for a subtopic
def generate_content(subtopic):
    content = content_chain.predict(subtopic=subtopic)
    return {
        "title": subtopic,
        "content": content
    }

# Generate contents for each subtopic in every module using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    for module in data.values():
        subtopics = module["Subtopics"]
        # Create a list of futures for content generation
        futures = [executor.submit(generate_content, subtopic) for subtopic in subtopics.values()]
        # Retrieve the results from the futures and update the subtopics
        for future, (subtopic_number, _) in zip(futures, subtopics.items()):
            content = future.result()
            subtopics[subtopic_number] = content

# Save the updated module_output to a JSON file
with open("module_output.json", "w") as file:
    json.dump(data, file, indent=4)

# Print the updated module_output
print(json.dumps(data, indent=4))


split content in slidee

In [ ]:
import json
import concurrent.futures

# Define the content_prompt template
content_prompt = PromptTemplate(
    input_variables=["subtopic"],
    template="Give me a detailed description of {subtopic}. Write in bulleted form and make sure to use code examples so that any beginner-level student can understand."
)

slide_prompt = PromptTemplate(
    input_variables=["content"],
    template="distribute the content in {content} to N number of slides intelligently. the contents in the slides must be extracted from {contents} and please don't write heading on what can we include in the slides , just take the content from {content} and distribute them."
)



# Create the LLMChain instance for splitting content
split_chain = LLMChain(llm=llm, prompt = slide_prompt )

# Create the LLMChain instance for generating content
content_chain = LLMChain(llm=llm, prompt=content_prompt)

data = json.loads(module_output)


# Function to generate content for a subtopic
def generate_content(subtopic):
    # Generate the full content using content_chain
    full_content = content_chain.predict(subtopic=subtopic)
    # Split the full content into multiple parts
    splitted_content = split_chain.predict(content = full_content)
    return {
        "title": subtopic,
        "full content": full_content,
        "split content": splitted_content

    }

# Generate contents for each subtopic in every module using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    for module in data.values():
        subtopics = module["Subtopics"]
        # Create a list of futures for content generation
        futures = [executor.submit(generate_content, subtopic) for subtopic in subtopics.values()]
        # Retrieve the results from the futures and update the subtopics
        for future, (subtopic_number, _) in zip(futures, subtopics.items()):
            content = future.result()
            subtopics[subtopic_number] = content

# Save the updated module_output to a JSON file
with open("module_output.json", "w") as file:
    json.dump(data, file, indent=4)

# Print the updated module_output
print(json.dumps(data, indent=4))
